# How to do time profiling during training

User can fetch times in several manners depending on complexity of
required time profiling:

## Single epoch and total time

Simpliest way to fetch time of single epoch and complete training is to
use `engine.state.times["EPOCH_COMPLETED"]` and
`engine.state.times["COMPLETED"]`:

In [19]:
import time
from ignite.engine import Engine, Events

# Dummy data 
data = [1, 2, 3]

# For every datapoint, trainer will sleep for 1 second
trainer = Engine(lambda engine, batch: time.sleep(1))

@trainer.on(Events.EPOCH_COMPLETED)
def log_epoch_time():
    print(f"{trainer.state.epoch}: {trainer.state.times['EPOCH_COMPLETED']}")

@trainer.on(Events.COMPLETED)
def log_total_time():
    print(f"Total: {trainer.state.times['COMPLETED']}")

trainer.run(data=data, max_epochs=5)

1: 3.0028934478759766
2: 3.0037014484405518
3: 3.003610849380493
4: 3.0037147998809814
5: 3.003558397293091
Total: 15.019093036651611


State:
	iteration: 15
	epoch: 5
	epoch_length: 3
	max_epochs: 5
	output: <class 'NoneType'>
	batch: 3
	metrics: <class 'dict'>
	dataloader: <class 'list'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

For details, see [`State`](https://pytorch.org/ignite/v0.4.5/generated/ignite.engine.events.State.html#state).

## Basic time profiling

User can setup [`BasicTimeProfiler`](https://pytorch.org/ignite/v0.4.5/generated/ignite.contrib.handlers.time_profilers.BasicTimeProfiler.html#basictimeprofiler) to fetch times spent in data
processing, training step, event handlers:

In [20]:
import time
from ignite.engine import Engine, Events
from ignite.contrib.handlers import BasicTimeProfiler

# Dummy data 
data = [1, 2, 3]

# For every datapoint, trainer will sleep for 1 second
trainer = Engine(lambda engine, batch: time.sleep(1))

# Create an object of the profiler and attach an engine to it
profiler = BasicTimeProfiler()
profiler.attach(trainer)

@trainer.on(Events.EPOCH_COMPLETED(every=3))
def log_intermediate_results():
    profiler.print_results(profiler.get_results())

trainer.run(data=data, max_epochs=3)


 ----------------------------------------------------
| Time profiling stats (in seconds):                 |
 ----------------------------------------------------
total  |  min/index  |  max/index  |  mean  |  std

Processing function:
9.01026 | 1.00065/3 | 1.00144/2 | 1.00114 | 0.00022

Dataflow:
0.00089 | 0.00003/6 | 0.00022/1 | 0.00010 | 0.00007

Event handlers:
0.00040

- Events.STARTED: []
0.00001

- Events.EPOCH_STARTED: []
0.00004 | 0.00000/0 | 0.00003/2 | 0.00001 | 0.00001

- Events.ITERATION_STARTED: []
0.00004 | 0.00000/0 | 0.00002/2 | 0.00000 | 0.00000

- Events.ITERATION_COMPLETED: []
0.00011 | 0.00000/7 | 0.00003/1 | 0.00001 | 0.00001

- Events.EPOCH_COMPLETED: ['log_intermediate_results']
0.00007 | 0.00003/1 | 0.00004/0 | 0.00004 | 0.00001

- Events.COMPLETED: []
not yet triggered



State:
	iteration: 9
	epoch: 3
	epoch_length: 3
	max_epochs: 3
	output: <class 'NoneType'>
	batch: 3
	metrics: <class 'dict'>
	dataloader: <class 'list'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

For details, see [`BasicTimeProfiler`](https://pytorch.org/ignite/v0.4.5/generated/ignite.contrib.handlers.time_profilers.BasicTimeProfiler.html#basictimeprofiler)

## Event handlers time profiling

If you want to get time breakdown per handler basis then you can setup
[`HandlersTimeProfiler`](https://pytorch.org/ignite/v0.4.5/generated/ignite.contrib.handlers.time_profilers.HandlersTimeProfiler.html#ignite.contrib.handlers.time_profilers.HandlersTimeProfiler):

In [29]:
import time
from ignite.engine import Engine, Events
from ignite.contrib.handlers import HandlersTimeProfiler

# Dummy data 
data = [1, 2, 3]

# For every datapoint, trainer will sleep for 1 second
trainer = Engine(lambda engine, batch: time.sleep(1))

# Create an object of the profiler and attach an engine to it
profiler = HandlersTimeProfiler()
profiler.attach(trainer)   

@trainer.on(Events.EPOCH_COMPLETED(every=1))
def log_intermediate_results():
    profiler.print_results(profiler.get_results())

trainer.run(data=data, max_epochs=3)


----------------------------  -------------------  --------------  --------------  --------------  --------------  --------------  
Handler                       Event Name                 Total(s)      Min(s)/IDX      Max(s)/IDX         Mean(s)          Std(s)  
----------------------------  -------------------  --------------  --------------  --------------  --------------  --------------  
log_intermediate_results      EPOCH_COMPLETED       not triggered       None/None       None/None            None            None  
----------------------------  -------------------  --------------  --------------  --------------  --------------  --------------  
Total                                                         0.0                                                                  
----------------------------  -------------------  --------------  --------------  --------------  --------------  --------------  
Processing took total 3.00376s [min/index: 1.00049s/0, max/index: 1.00204s/

State:
	iteration: 9
	epoch: 3
	epoch_length: 3
	max_epochs: 3
	output: <class 'NoneType'>
	batch: 3
	metrics: <class 'dict'>
	dataloader: <class 'list'>
	seed: <class 'NoneType'>
	times: <class 'dict'>

For details, see [`HandlersTimeProfiler`](https://pytorch.org/ignite/v0.4.5/generated/ignite.contrib.handlers.time_profilers.HandlersTimeProfiler.html#ignite.contrib.handlers.time_profilers.HandlersTimeProfiler).

## Custom time measures

Custom time measures can be performed using [`Timer`](https://pytorch.org/ignite/v0.4.5/generated/ignite.handlers.timing.Timer.html#timer). See its
docstring for details.